# Universal Sentence Encoder Baseline for Madar task 1

In this notebook, we will walk you through the process of reproducing the Universal Sentence Encoder baseline for Madar task 1.

## Loading Required Modules

We start by loading the needed python libraries.

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import pandas as pd
import tensorflow_text
from tensorflow import keras
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

2021-08-27 22:49:44.400259: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-27 22:49:44.400274: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Loading Data

Using pandas, we can load and inspect the training, validation, and testing datasets as follows:

In [2]:
df_train = pd.read_csv("../../data/madar-1/MADAR-Corpus-26-train.tsv", sep="\t", header=None, names=["Text", "label"])
df_dev = pd.read_csv("../../data/madar-1/MADAR-Corpus-26-dev.tsv", sep="\t", header=None, names=["Text", "label"])
df_test = pd.read_csv("../../data/madar-1/MADAR-Corpus-26-test.tsv", sep="\t", header=None, names=["Text", "label"])

Below we list the 5 first entries in the training data.

In [3]:
df_train.head()

,Text,label
0,هناك ، أمام بيانات السائح تماما .,MSA
1,لم اسمع بهذا العنوان من قبل بالقرب من هنا .,MSA
2,استمر في السير في هذا الطريق حتى تجد صيدلية .,MSA
3,كم تكلفة الإفطار ؟,MSA
4,كيف أستطيع مساعدتك ؟,MSA


Below we list the 5 first entries in the development data.

In [4]:
df_dev.head()

,Text,label
0,بالمناسبة ، اسمي هيروش إيجيما .,MSA
1,"هذا القطار يتوقف في لاك فورست , أليس كذلك ؟",MSA
2,"هذا الكارت , حسناً ؟",MSA
3,لم يخرج من الماكينة شيء .,MSA
4,عندك أية شيء يمكن أن أتعاطه للطفح الجلدي ؟,MSA


Below we list the 5 first entries in the test data.

In [5]:
df_test.head()

,Text,label
0,لا أعرف كثيراً عن النبيذ ؟ ماذا يناسب هذا الطبق ؟,MSA
1,رايح عالمدرسة هون ؟,DAM
2,قهوه مع كريمة و سكر ، لوسمحت .,SAN
3,بأي محطة لازم أنزل عشان أروح على امباير ستيت ب...,AMM
4,اسمي ميتشيكو تاناكا ، ورقم الرحلة خمسة صفر واح...,JED


## Model Preparation

We start by setting the randomisation seed:

In [6]:
tf.random.set_seed(123)

Next we load the Universal Sentence Encoder (WARNING: This will download and cache a huge model of around 1 GB in size)

In [7]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

2021-08-27 22:49:50.366307: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-27 22:49:50.366327: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-27 22:49:50.366340: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cadmus-ERAZER-P6605-MD61363): /proc/driver/nvidia/version does not exist
2021-08-27 22:49:50.366460: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-27 22:49:51.513326: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] Non

Then we define the input and output to the model:

In [8]:
sentence = keras.Input(shape=512, name='sentence')
label = keras.Input(shape=(1,), name='label')

This is followed by defining the structure of the network:

In [9]:
logits = keras.layers.Dense(512)(sentence)
logits = keras.layers.Dense(26, activation=tf.nn.softmax)(logits)

Then we construct and compile the model:

In [10]:
model = keras.Model(sentence, outputs=logits)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Model Training

First we perpare the inputs and outputs to be fed to the model during training:

In [11]:
le = LabelEncoder()
le.fit(df_train["label"])
Y_train = le.transform(df_train["label"])
X_dev = embed(df_dev["Text"])
Y_dev = le.transform(df_dev["label"])

Given the size of the input we need to constructor a generator as follows:

In [14]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, text, labels, batch_size=32, shuffle=True):
        'Initialization'
        
        self.batch_size = batch_size
        self.labels = labels
        self.text = text
        self.shuffle = shuffle
        self.list_IDs = list_IDs
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_text_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_text_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_text_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = []
        y = []

        for i in list_text_temp:
            y.append(self.labels[i])
            X.append(self.text[i])
        return embed(X), np.array(y)

Next we fit the data:

In [17]:
training_generator = DataGenerator(df_train.reset_index()["index"].tolist(), df_train.Text, Y_train)
model.fit(training_generator, epochs=5, validation_data = (X_dev, Y_dev))

Epoch 1/5
1300/1300 [==============================] - 25s 19ms/step - loss: 2.8872 - accuracy: 0.1663 - val_loss: 2.7070 - val_accuracy: 0.2129
Epoch 2/5
1300/1300 [==============================] - 26s 20ms/step - loss: 2.6911 - accuracy: 0.2180 - val_loss: 2.6604 - val_accuracy: 0.2227
Epoch 3/5
1300/1300 [==============================] - 31s 24ms/step - loss: 2.6446 - accuracy: 0.2307 - val_loss: 2.6427 - val_accuracy: 0.2329
Epoch 4/5
1300/1300 [==============================] - 29s 22ms/step - loss: 2.6197 - accuracy: 0.2375 - val_loss: 2.6300 - val_accuracy: 0.2363
Epoch 5/5
1300/1300 [==============================] - 29s 22ms/step - loss: 2.6042 - accuracy: 0.2434 - val_loss: 2.6233 - val_accuracy: 0.2398


## Submission Preperation

We perpare the features for each testset instance as follows:

In [18]:
X_test = embed(df_test["Text"])
Y_test = le.transform(df_test["label"])

Then we predict the labels for each and evaluate the f1 score:

In [19]:
predictions = np.argmax(model.predict(X_test), 1)
f1_score(Y_test, predictions, average="macro")

0.22385777237805732

We perpare the predictions as a pandas dataframe.

In [20]:
df_preds = pd.DataFrame(data=le.inverse_transform(predictions))

In the final step, we save the predictions as required by the competition guidelines.

In [21]:
if not os.path.exists("predictions"):
    os.mkdir("predictions")
df_preds.to_csv("./predictions/madar.tsv", index=False, header=False, sep="\t")